(case_studies)=
# Case studies

Description of the test cases
* *Motivate why they were selected as the candidates*
* *Where the data come from, measurement accuracy/uncertainty*
* *Describe the differences between the test cases to show how generally applicable the methodology is*
* *Success criteria: make a statement about which deviations or accuracy in the prediction are accepted as ''successful'' and why*


The proposed PIT is studied for two test cases: wPCC (see [fig](wpcc_mdl)) and the well known KVLCC2 test case.

The wPCC test case is a wind powered car carrier tested at SSPA. This twin screw ship with large rudders has good course stabilility and is expected to have almost linear and symmetric hydrodynamic manoeuvring forces. The test setup and measurement procedures at SSPA is also well known to the authors which all in all makes it ideal as a first test case of the PIT. 

```{image} figures/wpcc_mdl.png
:alt: wpcc_mdl
:class: bg-primary mb-1
:width: 200px
:align: center
```

KVLCC2 test case was selected as the second test case. It is a known test case and model test data from HSVA and MARIN was made available from SIMMAN2008 conference {cite:p}`stern_experience_2011`. This single screw ship is more course unstable than the wPCC test case and is therefore expected to have more nonlinear hydrodynamic manoeuvring forces which are also unsymetrical due to the single propeller. This makes it good as the seconds test case with PIT on a more nonlinear and unsymetrical model.

The general aim with developing a VMM with PIT is to be able to make predictions for unseen data, outside the known data from model tests or full scale operation. The aim with both test cases is therefore to predict turning circle manouvres with VMM:s that are trained on model tests data where turning circle manoeuvres are excluded. The training data contains drift angles and yaw rates that are much smaller than what is encounterd during a turning circle manouvre.

Main dimensions of the test cases can be seen in table [tab](Main_dimensions_of_test_case_ship_models).

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters
from jb_helpers import df_to_myst, parameter_to_latex

In [ ]:
model_test_ids.keys()

In [ ]:
ships = ['wpcc','kvlcc2_hsva']

ship_datas = {}
initial_parameters = {}

for ship in ships:
       
    ship_datas[ship] = catalog.load(f"{ship}.ship_data")
    initial_parameters[ship] = catalog.load(f"{ship}.initial_parameters")
    

ship_datas = pd.DataFrame(ship_datas)
ship_datas.dropna(how='any', inplace=True)

initial_parameters = pd.DataFrame(initial_parameters)
initial_parameters.drop(index='Xthrust', inplace=True)
mask = (initial_parameters!=0).all(axis=1)
initial_parameters = initial_parameters.loc[mask].copy()

In [ ]:
ship_data_table = ship_datas.copy()
ship_data_table.rename(columns={'wpcc':'WPCC', 'kvlcc2_hsva': 'KVLCC2 (HSVA)'}, inplace=True)
ship_data_table = ship_data_table.transpose()
ship_data_table.drop(columns=['rho'], inplace=True)
ship_data_table.rename(
    columns={
        'T':'T [m]',
        'L':'L [m]',
        'B':'B [m]',
        'x_G': 'LCG [m]',
        'm': 'm [kg]',
        'I_z':'Izz [kg m2]',
        'volume':'Disp [m3]',
        'scale_factor':'scale factor',
        'TWIN':'Np',
    },
    
    inplace=True)

ship_data_table['Np']+=1
ship_data_table['Np']=ship_data_table['Np'].astype(int)

ship_data_table = ship_data_table.round(decimals=2)
#glue("ship_datas", ship_data_table)

print(df_to_myst(ship_data_table, title="Main dimensions of test case ship models"))
print('_____________')

```{list-table} This table title
:header-rows: 1
:name: This_table_title
* -  
  - T [m]
  - L [m]
  - CB
  - B [m]
  - LCG [m]
  - m [kg]
  - Izz [kg m2]
  - Disp [m3]
  - scale factor
  - x_r
  - Np
  - x_p
  - D
  - w_p0
* - WPCC
  - 0.21
  - 5.01
  - 0.45
  - 0.95
  - 0.0
  - 441.03
  - 693.12
  - 0.44
  - 41.2
  - -2.42
  - 2
  - -2.42
  - 0.12
  - 0.15
* - KVLCC2 (HSVA)
  - 0.46
  - 7.0
  - 0.81
  - 1.27
  - 0.24
  - 3272.0
  - 9908.58
  - 3.27
  - 45.7
  - -3.5
  - 1
  - -3.39
  - 0.2
  - 0.4

```

### Initial conditions and assumptions

In [ ]:
s = ""
mask = df_parameters['brix'].notnull()
df_brix = df_parameters.loc[mask].copy()
df_brix.sort_index(inplace=True)
for index, row in df_brix.iterrows():
    
    eq = sp.Eq(row['symbol'],row['brix'])
    name = f"eq_{index}"
    glue(name,eq)
    
    s+=f"""
```{{glue:math}} {name}
:label: "{name}"
```
    """
print(s)   

The PIT requires an initial guessed linear model. Hydrodynamic derivatives for these inital models for the two test cases are shown in table [tab](intial_guess). These are calculated with semi empirical formulas ([eq](eq_Nr)- [eq](eq_Yvdot)) taken from {cite:p}`brix_manoeuvring_1993`. 


```{glue:math} eq_Nr
:label: "eq_Nr"
```
    
```{glue:math} eq_Nrdot
:label: "eq_Nrdot"
```
    
```{glue:math} eq_Nv
:label: "eq_Nv"
```
    
```{glue:math} eq_Nvdot
:label: "eq_Nvdot"
```
    
```{glue:math} eq_Xudot
:label: "eq_Xudot"
```
    
```{glue:math} eq_Yr
:label: "eq_Yr"
```
    
```{glue:math} eq_Yrdot
:label: "eq_Yrdot"
```
    
```{glue:math} eq_Yv
:label: "eq_Yv"
```
    
```{glue:math} eq_Yvdot
:label: "eq_Yvdot"
```
    


In [ ]:
initial_parameter_table = initial_parameters.rename(columns={'wpcc':'WPCC', 'kvlcc2_hsva': 'KVLCC2 (HSVA)'})
initial_parameter_table = initial_parameter_table.transpose()
initial_parameter_table.columns = [parameter_to_latex(x) for x in initial_parameter_table.columns]

#glue("initial_parameters", 1000*initial_parameter_table)

print(df_to_myst((1000*initial_parameter_table).round(decimals=3), 
                 title="Initial guessed derivatives in linear models (times 1000)", 
                 name="intial_guess"))


```{list-table} Initial guessed derivatives in linear models (times 1000)
:header-rows: 1
:name: intial_guess
* -  
  - $ N_{\delta} $
  - $ N_{r} $
  - $ N_{\dot{r}} $
  - $ N_{v} $
  - $ N_{\dot{v}} $
  - $ X_{\dot{u}} $
  - $ Y_{\delta} $
  - $ Y_{r} $
  - $ Y_{\dot{r}} $
  - $ Y_{v} $
  - $ Y_{\dot{v}} $
* - WPCC
  - -1.5
  - -1.719
  - -0.299
  - -3.184
  - -0.128
  - 0.179
  - 3.0
  - 2.402
  - -0.303
  - -9.713
  - -6.109
* - KVLCC2 (HSVA)
  - -1.5
  - -3.415
  - -0.822
  - -8.707
  - -1.166
  - 1.05
  - 3.0
  - 4.305
  - -1.271
  - -25.266
  - -15.846

```

Hydrodynamic deriviatives in the Abkowitz model are identified with linear regression on forces and moments from the model tests inverse dynamics. A comparison between the inverse dynamic forces and prediction with the regressed model is shown for one of the KVLCC2 model tests in [fig](force_prediction).

```{glue:figure} force_prediction
:name: "force_prediction"

Forces and moment from model test inverse dynamics and prediction with the regressed model.
```


### Motion regression

In [ ]:
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force

vmm_name = 'vmm_abkowitz'
id = 'HSVA_CPMC_KVLCC2_Z_20_10_S'
ship = 'kvlcc2_hsva'

data_ek_smooth = catalog.load(f"{ship}.initial.{id}.data_ek_smooth")
added_masses = catalog.load(f"{ship}.added_masses")
ship_parameters = catalog.load(f"{ship}.ship_data")
vmm = catalog.load(vmm_name)
regression = catalog.load(f"{ship}.initial.{vmm_name}.joined.regression")
df_force = predict_force(data=data_ek_smooth, added_masses=added_masses, ship_parameters=ship_parameters, vmm=vmm) 

* *If needed: detailed description of each test case and ship model*